# Introduction of analog Hamiltonian simulation

Analog Hamiltonian Simulation (AHS) is a different paradigm of quantum computing compared to the gate-based quantum computing. The idea of AHS is to use a well-controlled quantum system and engineer its interaction to mimic the dynamics of the Hamiltonian of the quantum system in question. 

In the gate-based quantum computation, the program is a quantum circuit consisting of a series of quantum gates, each of which acts only a small subset of qubits. In contrast, an AHS program is one or a sequence of time-dependent Hamiltonians that govern the time evolution of all the qubits. The comparison can be seen in the following figure (Source: <a href="https://arxiv.org/abs/2006.12326">Henriet et al.</a>), where the left side shows a typical quantum circuit, and the right side illustrates that, during AHS, the effect of the evolution under a Hamiltonian can be understood as a unitary acting simultaneously on all qubits.

![Comparison_AHS.png](Comparison_AHS.png)

In this notebook, we focus on running Analog Hamiltonian Simulations with Rydberg atoms. 

# Running AHS with Rydberg atoms

An Analog Hamiltonian Simulation program is fully specified by its quantum register and (time-dependent) Hamiltonian.

In [1]:
from braket.ahs.hamiltonian import Hamiltonian
from braket.ahs.atom_arrangement import AtomArrangement
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation

register = AtomArrangement()
H = Hamiltonian()

ahs_program = AnalogHamiltonianSimulation(
    hamiltonian=H,
    register=register
)

In order to run AHS program with Rydberg atoms, we will first have to understand what type of Hamiltonian can the Rydberg atoms simulate.

## Introduction to Rydberg Hamiltonian

Depending on the atomic states we use for the Rydberg system, its Hamiltonian could take different forms. Here we shall focus on the following type of Hamiltonian

\begin{align}
H(t) = \sum_{k=1}^N H_{\text{drive}, k}(t) + \sum_{k=1}^N H_{\text{shift}, k}(t) + \sum_{j=1}^N\sum_{k = j}^N H_{\text{vdW}, j, k},
\end{align}

where $j, k=1,2,\ldots N$ index the atoms in the program register. We describe the nature and effect of each term in the following sections.

### Register

But first, we need to define a quantum register on which this Hamiltonian will act: a 2-dimentional layout of neutral atoms.

In [2]:
import numpy as np

# e.g. three atoms in an equilateral triangle with 5 edges
a = 5e-6

register.add([0, 0])
register.add([a, 0.0])
register.add([0.5 * a, np.sqrt(3)/2 * a]);

### Driving field

The first term of the Hamiltonian represents the effect of a **driving field** that addresses the atoms simultaneously and uniformly

\begin{align}
H_{\text{drive}, k}(t) = \frac{\Omega(t)}{2}\left(e^{i\phi(t)}\sigma_k + e^{-i\phi(t)}\sigma_k^\dagger\right) - \Delta_\text{global}(t)n_k,
\end{align}

where $\Omega$, $\phi$, and $\Delta_\text{global}$ to denote the **amplitude** (Rabi frequency), laser **phase**, and the **detuning** of the driving laser field. Here $\sigma_k = |g_k\rangle\langle r_k|$ is the lowering operator, and $n_k = |r_k\rangle\langle r_k|$ is the number operator of atom $k$; the kets $|g_k\rangle$ and $|r_k\rangle$ denote the ground and Rydberg states, respectively. The $\Omega$ part of the driving term is identical to a uniform (time-dependent) _transverse_ magnetic field, whereas the $\Delta_\text{global}$ part implements the effect of a _longitudinal_ magnetic field, in a spin-model representation.

In [3]:
from braket.ahs.time_series import TimeSeries
from braket.ahs.driving_field import DrivingField

# e.g. trapzoid amplitude time series
t_max = 4e-6  # seconds
t_ramp = 1e-7  # seconds 
omega_max = 6.3e6  # rad / s
omega = TimeSeries()
omega.put(0.0, 0.0)
omega.put(t_ramp, omega_max)
omega.put(t_max - t_ramp, omega_max)
omega.put(t_max, 0.0)

# e.g. all-zero phase and detuning
phi = TimeSeries().put(0.0, 0.0).put(t_max, 0.0)  # (time [s], value [rad/s])
delta_global = TimeSeries().put(0.0, 0.0).put(t_max, 0.0)  # (time [s], value [rad])

drive = DrivingField(
    amplitude=omega,
    phase=phi,
    detuning=delta_global
)

H += drive

### Shifting field

The second term in $H(t)$ represents the effect of a **shifting field** that detunes atoms according to a non-uniform pattern.
\begin{align}
H_{\text{shift}, k}(t) = -\Delta_\text{local}(t)h_k \,n_k,
\end{align}

where $\Delta_\text{local}(t)$ is the time-dependent **magnitude** of the frequency shift, and $h_k$ is the atom-dependent **pattern**, which is a dimensionless number between 0 and 1. This shifting term is identical to a non-uniform (and time-dependent) _longitudinal_ magnetic field in a spin-model representation.


In [4]:
from braket.ahs.field import Field
from braket.ahs.pattern import Pattern
from braket.ahs.shifting_field import ShiftingField

# e.g. triangular magnitude time series
delta_local = TimeSeries().put(0.0, 0.0).put(t_max /2, 2e7).put(t_max, 0.0)

# e.g. 100% shift for the atoms on bottom of the triangular register
#      and 50% shift for the atom on the top
h = Pattern([1, 1, 0.5])

shift = ShiftingField(
    magnitude=Field(
        time_series=delta_local,
        pattern=h
    )
)

H += shift

### Rydberg-Rydberg interaction

Finally, the third term in $H(t)$ is the van de Waals interaction between all pairs of Rydberg atoms,

\begin{align}
H_{\text{vdW}, j, k} = \frac{C_6}{R_{j,k}^6} \,n_j\, n_k,
\end{align}

where $C_6$ is a fixed interaction coefficient, and $R_{j,k}=|{\bf x}_j-{\bf x}_k|$ is the distance between atoms $j$ and $k$. This interacting shifts the frequency of the Rydberg level of all atoms that are close to an atom that is already in its Rydberg state. While the overall coefficient, $C_6$ is a fixed value (determined by the nature of the ground and Rydberg states), the strength of this interaction can be tuned by adjusting the pairwise distance $R_{j,k}$ between atoms.

**Introduction to Rydberg blockade**

For the interaction coefficient, the value of $C_6$ depends on the atom species, and the states used in the simulation. Here we shall take the value 
\begin{align}
C_6 = 5.42\times 10^{-24} \text{rad} m^6/s
\end{align}
for $|r\rangle = |70S_{1/2}\rangle$ of the $^{87}$Rb atoms. For the typical scenario, where atoms are separated by $4\times10^{-6}$ meters, the Van del Waals interaction reads $V_{jk}=1.32\times10^9 \text{rad}/s$, which is much larger than the typical scale of the Rabi frequency (around $6\times10^6 \text{rad}/s$). As a result, when the separation of two atoms is within certain distance, it is nearly impossible to drive them to the Rydberg state simultaneously. 

This is called the Rydberg blockade phenomena, illustrated in the figure below where $R$ is the separation between the atoms, and $E$ indicates the energies or frequencies of the different two-atom states as $R$ changes. The vertical arrows indicate the effect of a uniform driving field (of with Rabi frequency $\Omega$) that successfully transitions the atoms from the $|00\rangle$ ground state to the 1-atom excited state $|\psi_+\rangle = (|0r\rangle + |r0\rangle)/\sqrt{2}$ (independent of $R$), but fails to get from there to the doubly-excited state $|rr\rangle$, if $R$ is smaller than $R_b = (C_6 / \Omega)^{1/6}$, the blockade radius.

<img src="Blockade.png" alt="drawing" style="width:250px;"/>

In [5]:
# Note:
# The van der Waals interaction term is implicitly assumed in the current version of the AHS module,
# its strength (C6 / R_{j,k}^6) is calculated (if using the local simulator) from the atomic positions 
# of the register, hence there is no need to specify it explicitly.

### Full program

The fully-specified program can be inspected with the following command.

In [6]:
ahs_program.to_ir().dict()

{'braketSchemaHeader': {'name': 'braket.ir.ahs.program', 'version': '1'},
 'setup': {'atomArray': {'sites': [[Decimal('0'), Decimal('0')],
    [Decimal('0.000005'), Decimal('0.0')],
    [Decimal('0.0000025'), Decimal('0.000004330127018922193')]],
   'filling': [1, 1, 1]}},
 'hamiltonian': {'drivingFields': [{'amplitude': {'sequence': {'values': [Decimal('0.0'),
       Decimal('6300000.0'),
       Decimal('6300000.0'),
       Decimal('0.0')],
      'times': [Decimal('0.0'),
       Decimal('1E-7'),
       Decimal('0.0000039'),
       Decimal('0.000004')]},
     'pattern': 'uniform'},
    'phase': {'sequence': {'values': [Decimal('0.0'), Decimal('0.0')],
      'times': [Decimal('0.0'), Decimal('0.000004')]},
     'pattern': 'uniform'},
    'detuning': {'sequence': {'values': [Decimal('0.0'), Decimal('0.0')],
      'times': [Decimal('0.0'), Decimal('0.000004')]},
     'pattern': 'uniform'}}],
  'shiftingFields': [{'magnitude': {'sequence': {'values': [Decimal('0.0'),
       Decimal('200000